#Customer Analytics using Deep Learning

* **Background information**
An Audio book company wants to be able to predict whether customers given 2 years trial period will convert in the next 6 months after trial period.

* **Metrics of Success**
Developing a fully functioning high accuracy deep learning model that can be able to predict whether a subscriber will convert or not.

In [1]:
#Importing the required prerequisites

#Import pandas for data manipulation
import pandas as pd

#Import pandas for scientific computation
import numpy as np

#Import StandardScaler to standardize the values
from sklearn.preprocessing import StandardScaler

#Import pickle to save models
import pickle

#Import tensorflow
import tensorflow as tf

##Model preprocessing

###Extract data from CSV

In [2]:
#Load data in an array form
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

#extract inputs exluding first and last columns
unscaled_inputs_all = raw_csv_data[:, 1:-1]

#Extract targets
targets_all = raw_csv_data[:, -1]

###Balance the dataset

In [3]:
num_one_targets = int(np.sum(targets_all))

zero_targets_counter = 0
indices_to_remove = []

#create a for loop
for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_targets_counter += 1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

###Standardize inputs
This will help the model perform better

In [4]:
#Give the scaler an identifier
scaler_deep_learning = StandardScaler()

#Fit the scaler to the balanced input data
scaled_inputs = scaler_deep_learning.fit_transform(unscaled_inputs_equal_priors)

###Shuffle the data

In [5]:
#Data might be ordered by date, hence become homogeneous at certain points
#We'll need to shaffle the data to make it as randomized as possible so that it
#won't affect the gradient descent

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

###Split the dataset into train, validation and test

In [6]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

#split datasets into inputs and targets
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

#Checking whether the target values are half or 50% of the total
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1821.0 3579 0.5088013411567477
218.0 447 0.48769574944071586
198.0 448 0.4419642857142857


The proportions of classes are almost equal at around 50%

###Saving the three datasets in *.npz
This is to ensure they are conversant with tensorflow

In [7]:
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

###Saving the Stardadization Scaler

In [8]:
pickle.dump(scaler_deep_learning, open('scaler_deep_learning.pickle', 'wb'))

##Model Outlining

###Loading data

In [15]:
#train data
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float64)
train_targets = npz['targets'].astype(np.int64)

#validation data
npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.int64)

#test data
npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.int64)

###Setting up layers


In [16]:
#input_size = 10 (predictors)
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             ])

###Model compiling

In [17]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

batch_size = 100
max_epochs = 100

#Early stopping controls overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

#Fitting the model
model.fit(
    train_inputs,
    train_targets,
    batch_size = batch_size,
    epochs = max_epochs,
    callbacks = [early_stopping],
    validation_data = (validation_inputs, validation_targets),
    verbose = 2
    )


Epoch 1/100
36/36 - 1s - loss: 6.1932 - accuracy: 0.1676 - val_loss: 3.3455 - val_accuracy: 0.3915 - 624ms/epoch - 17ms/step
Epoch 2/100
36/36 - 0s - loss: 1.7032 - accuracy: 0.4395 - val_loss: 0.8683 - val_accuracy: 0.4631 - 98ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.7542 - accuracy: 0.5155 - val_loss: 0.6796 - val_accuracy: 0.5436 - 112ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.6402 - accuracy: 0.6024 - val_loss: 0.5997 - val_accuracy: 0.6331 - 103ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.5811 - accuracy: 0.6778 - val_loss: 0.5392 - val_accuracy: 0.7025 - 107ms/epoch - 3ms/step
Epoch 6/100
36/36 - 0s - loss: 0.5386 - accuracy: 0.7404 - val_loss: 0.4974 - val_accuracy: 0.7763 - 102ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.5030 - accuracy: 0.7974 - val_loss: 0.4580 - val_accuracy: 0.8054 - 105ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.4682 - accuracy: 0.8248 - val_loss: 0.4270 - val_accuracy: 0.8188 - 98ms/epoch - 3ms/step
Epoch 9/1

The model has an accuracy of 89% on validation set

##Testing the model

In [18]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)
print('\nTest loss: {0:.2f}. Test accurac  y: {1:.2f}%'.format(test_loss, test_accuracy*100.))

14/14 [==============================] - 0s 2ms/step - loss: 0.3083 - accuracy: 0.9040

Test loss: 0.31. Test accurac  y: 90.40%


The model performed much better on the test set with a 90% accuracy

In [19]:
#Obtaining the probability of classes

np.argmax(model.predict(test_inputs), axis = 1)

array([1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,

###Saving the model

In [20]:
model.save('audiobooks_model.h5')

##Predicting new data

Let's assume we want to use the model to predict new instances.

###Import prerequisites

In [21]:
import numpy as np
import tensorflow as tf
import pickle

###Load the scaler and the model

In [22]:
scaler_deep_learning = pickle.load(open('scaler_deep_learning.pickle','rb'))
model = tf.keras.models.load_model('audiobooks_model.h5')

###Load new data

In [23]:
raw_data = np.loadtxt('New_Audiobooks_Data.csv', delimiter = ',')
new_data_inputs = raw_data[:,1:]

###Predict the probability of  customer to convert

In [24]:
new_data_inputs_scaled = scaler_deep_learning.transform(new_data_inputs)

In [30]:
pred = np.argmax(model.predict(new_data_inputs_scaled), 1)
pred

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1])